In [1]:
# !pip install pandas
# !pip install sqlalchemy
# CSV --> SQL RAM --> SQL --> Results

In [2]:
import pandas as pd
df = pd.read_csv('penguins.csv')
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE


In [4]:
from sqlalchemy import create_engine, text

In [5]:
temp_db = create_engine('sqlite:///:memory:',echo=True)

In [6]:
data = df.to_sql(name='Penguins',con=temp_db)

2023-10-16 12:18:20,113 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Penguins")
2023-10-16 12:18:20,114 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-16 12:18:20,115 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Penguins")
2023-10-16 12:18:20,116 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-10-16 12:18:20,118 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-16 12:18:20,119 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Penguins" (
	"index" BIGINT, 
	species TEXT, 
	island TEXT, 
	bill_length_mm FLOAT, 
	bill_depth_mm FLOAT, 
	flipper_length_mm FLOAT, 
	body_mass_g FLOAT, 
	sex TEXT
)


2023-10-16 12:18:20,121 INFO sqlalchemy.engine.Engine [no key 0.00096s] ()
2023-10-16 12:18:20,122 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Penguins_index" ON "Penguins" ("index")
2023-10-16 12:18:20,122 INFO sqlalchemy.engine.Engine [no key 0.00045s] ()
2023-10-16 12:18:20,123 INFO sqlalchemy.engine.Engine COMMIT
2023-10-16 12:18:20,125 INFO sqlalchemy.engine.Engine BE

In [13]:
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT * FROM Penguins"))

2023-10-16 12:23:49,232 INFO sqlalchemy.engine.Engine SELECT * FROM Penguins
2023-10-16 12:23:49,233 INFO sqlalchemy.engine.Engine [cached since 268.2s ago] ()


In [15]:
# result.all()

In [10]:
# NLP -> SQL -> CONTEXT -> SELECT col_name FROM table_name properties

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


In [16]:
def create_prefix(query):
    # query = input("What is your question about the penguins table?")
    prefix = f"""Return a SQL statement that answers the following query:
    {query}
    
    For a table called 'Penguins" with the following properties:
    #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
     0   species            344 non-null    object 
     1   island             344 non-null    object 
     2   bill_length_mm     342 non-null    float64
     3   bill_depth_mm      342 non-null    float64
     4   flipper_length_mm  342 non-null    float64
     5   body_mass_g        342 non-null    float64
     6   sex                333 non-null    object 
     
    Example Rows:
    (0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
    (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')
    
    Only return the SQL statement for the query.
    """
    return prefix

In [17]:
def user_input():
    query = input("Ask a question about the Penguins table")
    return create_prefix(query)

In [18]:
prefix = user_input()

Ask a question about the Penguins table How many penguins are there in the table?


In [20]:
print(prefix)

Return a SQL statement that answers the following query:
    How many penguins are there in the table?
    
    For a table called 'Penguins" with the following properties:
    #   Column             Non-Null Count  Dtype  
    ---  ------             --------------  -----  
     0   species            344 non-null    object 
     1   island             344 non-null    object 
     2   bill_length_mm     342 non-null    float64
     3   bill_depth_mm      342 non-null    float64
     4   flipper_length_mm  342 non-null    float64
     5   body_mass_g        342 non-null    float64
     6   sex                333 non-null    object 
     
    Example Rows:
    (0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
    (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')
    
    Only return the SQL statement for the query.
    


In [21]:
from vertexai.language_models import CodeGenerationModel

In [22]:
codegen_model = CodeGenerationModel.from_pretrained('code-bison')

In [26]:
print("Hello, I am your AI database assistant")
print('\n\n')
prefix = user_input()
result = codegen_model.predict(prefix=prefix)

Hello, I am your AI database assistant





Ask a question about the Penguins table How many rows are in the database?


In [28]:
result.text

'```sql\nSELECT COUNT(*)\nFROM Penguins;\n```'

In [29]:
def clean_sql(sql_statement):
    return sql_statement.replace('```sql','').replace("```","").replace('\n',' ')

In [30]:
clean_sql(result.text)

' SELECT COUNT(*) FROM Penguins; '

In [36]:
def nlp_assistant():
    print("Hello, I am your AI database assistant")
    print('\n\n')
    prefix = user_input()
    result = codegen_model.predict(prefix=prefix)
    sql = clean_sql(result.text)
    with temp_db.connect() as conn:
        print('\n Ok I am running this SQL Statement:')
        result = conn.execute(text(sql))
    print('\n\n\n==========================================')
    print("Here are your results:")
    print(result.all())

In [39]:
nlp_assistant()

Hello, I am your AI database assistant





Ask a question about the Penguins table What are the unique island names?



 Ok I am running this SQL Statement:
2023-10-16 12:42:39,164 INFO sqlalchemy.engine.Engine  SELECT DISTINCT island FROM Penguins; 
2023-10-16 12:42:39,164 INFO sqlalchemy.engine.Engine [generated in 0.00046s] ()



Here are your results:
[('Torgersen',), ('Biscoe',), ('Dream',)]
